In [ ]:
# ! pip install git+https://git@github.com/ping/instagram_private_api.git@1.6.0 --upgrade
# ! pip install instaloader

In [1]:
from instagram_private_api import Client, ClientCompatPatch
from instagram_private_api.errors import ClientError  
import time
class Instagram(object):
    def __init__(self, username = None, password = None):
        self.username = username
        self.password = password
        self.api = self.login()
        
    def login(self):
        return Client(self.username, self.password)
    
    def get_userId(self, user):
        # Getting userId to send follo or unfollow request according to this python package !
        ''' return userId for crossponding user which can be used to send follow | unfollow request '''
        return self.api.user_detail_info(user)['reel_feed']['id']
    
    def get_mediaId(self, user):
        ''' Returns mediaId for public account, 
            Note: Only top 12 media id is returned if total number of media is more than 12 else return media id
            of all the posts
        '''
        return [item['id'] for item in self.api.user_detail_info(user)['feed']['items']]
    
    def follow(self, user):
        userId = self.api.user_detail_info(user)['reel_feed']['id']
        self.api.friendships_create(userId)
        
    def unfollow(self, user):
        userId = self.api.user_detail_info(user)['reel_feed']['id']
        self.api.friendships_destroy(userId) 
        
    def likeUserPhotos(self, user, amount = None, sleepTime = 2):
        '''
        Like photos of user if username is given.
        
        # Note : change sleepTime if required
        
        Example: 
            >>> likeUserPhotos(user, amount = None, sleepTime = 2)
        '''
        media_id = [item['id'] for item in self.api.user_detail_info(user)['feed']['items']]
        if amount == None : 
            count = len(media_id)
        elif amount > len(media_id):
            count = len(media_id)
        else:
            count = amount
        for i in range(0 , count):
            self.api.post_like(media_id[i])
            time.sleep(sleepTime)
    def privateAccount(self, user):
        ''' True if the account is private else False '''
        return self.api.user_detail_info(user)['user_detail']['user']['is_private']
    
    def following(self, user):
        '''return bool value, False if I am not following that user else True '''
        return self.api.user_detail_info(user)['user_detail']['user']['friendship_status']['following']

    def followed_by(self, user):
        '''return bool value, False if I am not followed by that user else True'''
        return self.api.user_detail_info(user)['user_detail']['user']['friendship_status']['followed_by']


In [2]:
import pandas as pd
import numpy as np
from joblib import load

class featureExtraction():
    def __init__(self, dataframe):
        '''   '''
        self.username = dataframe['username'].values
        self.post = dataframe['post'].values
        self.privacy = dataframe['privacy'].values
        self.following = dataframe['followings'].values
        self.followers = dataframe['followers'].values
        self.ln_followings = np.log(dataframe['followings'].values)
        self.ln_followers = np.log(dataframe['followers'].values)
        
        # For modelling we use output. While using model disable it
        # self.output = dataframe['Output'].values

    def _following(self):
        """ categorically dividing following into  """
        category = []
        for count in self.following :
            if count > 10000 :
                category.append(5)
            elif count > 1000 :
                category.append(4)
            elif count > 500:
                category.append(3)
            elif count > 100:
                category.append(2)
            else:
                category.append(1)
        return category

    def _follower(self):
        """ return the category  """
        category = []
        for count in self.followers :
            if count > 10000 :
                category.append(5)
            elif count > 1000 :
                category.append(4)
            elif count > 500:
                category.append(3)
            elif count > 100:
                category.append(2)
            else:
                category.append(1) 
        return category

    def _post(self):
        """ return the category  """
        category = []
        for count in self.post :
            if count > 500 :
                category.append(5)
            elif count > 100 :
                category.append(4)
            elif count > 50:
                category.append(3)
            elif count > 10:
                category.append(2)
            elif count >= 1:
                category.append(1)
            else:
                category.append(0)
        return category

    def _privacy(self):
        ''' 
        2: Open Account
        1: Close Account
        0: unknown account
        '''
        category = []
        for privacy in self.post :
            if privacy == "Open" :
                category.append(2)
            elif privacy == "Private" :
                category.append(1)
            else:
                category.append(0)
        return category
    def _ratioFollowingFollower(self):
        ''' number of followers per followings'''
        return self.following/self.followers

    def exeFeatureExtraction(self):
        df = pd.DataFrame(data = self.ln_followings, columns = ['ln_following'])
        df['ln_follower'] = self.ln_followers
        df['post_category'] = self._post()
        df['privacy_category'] = self._privacy()
        df['follower_category'] = self._follower()
        df['following_category'] = self._following()
        df['ratio_Follower_Following'] = self._ratioFollowingFollower()
        
        # while modelling for a new user we use output, 
        # df['output'] = self.output
        return df

In [3]:
# insta = Instagram(username = 'rat163002', password = 'Abbi@163')
insta = Instagram(username = '_hernando_malik', password = 'Anand@1630')
# insta = Instagram(username = 'abbijeetanand', password = 'Abhi@1630')

In [8]:
model = load('model/lgm.model')
df = pd.read_csv('data/jesus22020-05-15.csv')
dataframe = df[df['private_account'] == False]
dataframe['private_account'] = 'Open'
dataframe['post'] = dataframe['post_media'] + dataframe['post_igtv']
data = dataframe.rename(columns = {'private_account': 'privacy', 'followees': 'followings'})

data = data[['username', 'privacy', 'post', 'followings', 'followers']]
data = data[(data.followings != 0) & (data.followers != 0) & (data.privacy == "Open")]

if len(data) == 0:
    pass
else:
    feature = featureExtraction(data)
    df = feature.exeFeatureExtraction()
    X = df.iloc[:, :7].values
    model_flag = model.predict(X)  #[0]
    
toFollow = pd.DataFrame()
toFollow['username'] = data['username']
toFollow['Flag'] = model_flag
toFollow = toFollow[toFollow['Flag'] == 0]

users = toFollow['username'].values

/home/abhijeet/anaconda3/envs/arblet/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/home/abhijeet/anaconda3/envs/arblet/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [ ]:
# time.sleep(650)
print('total users to follow here is ', len(users))
RequestSent = 0
privateAccount = 0
Error = 0
for user in users:
    try:
        if (insta.following(user) == False and insta.followed_by(user) == False ):
            time.sleep(2)
            insta.follow(user)
            time.sleep(2)

            insta.likeUserPhotos(user, amount = 4, sleepTime = 2)
            print('Sent Request to : ', user)
            time.sleep(4)
            RequestSent += 1
        else: 
            print('already know this user : ', user)

    except ClientError:
        print('Error in requesting : ', user)
        time.sleep(3)
        Error += 1

print('Total Request Sent :', RequestSent)
print('Total Error Happened:', Error)
print('Total Private Account:', privateAccount)
print(len(users))

total users to follow here is  5


/home/abhijeet/anaconda3/envs/arblet/lib/python3.7/site-packages/instagram_private_api/endpoints/users.py:44: ClientExperimentalWarning: This endpoint is experimental. Do not use.
  warnings.warn('This endpoint is experimental. Do not use.', ClientExperimentalWarning)


Error in requesting :  baby_sham_la
